In [1]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import rcParams

import seaborn as sns
from starfysh import (AA, utils, plot_utils, post_analysis)
from starfysh import starfysh as sf_model

In [2]:
import scanpy as sc
import leidenalg
import scipy
#import squidpy as sq
import anndata as ad

In [15]:
#load adata
adata = sc.read_h5ad("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/alos/colon_work/adata/star_example/106_A4-60_B3_0_adata.h5ad")

In [3]:
import scanpy as sc

In [20]:
data_path = '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/alos/colon_work/adata/'
sample_id = 'star_example'
sig_name = 'full_sigs.csv'
gene_sig = pd.read_csv(os.path.join(data_path, sig_name))

In [17]:
gene_sig = utils.filter_gene_sig(gene_sig, adata.to_df())
gene_sig.head()

,Unnamed: 0,T cells CD4 CCR7,T cells CD4 IL7R,T cells CD4 Tfh CXCL13,T regs FOXP3,B cells memory,B cells naive,NK,Macrophage M1,Macrophage M2,...,pDC,Terminal exhaustion,Precursor exhaustion,CAFs MSC iCAF-like,CAFs myCAF-like,PVL differentiated,PVL immature,Normal epithelial,Plasmablasts,Melanoma
0,0,CCR7,IL7R,CXCL13,TNFRSF4,MS4A1,CD79A,GNLY,C1QB,CXCL10,...,IL3RA,CD3G,TCF7,APOD,COL1A1,ACTA2,CCL19,KRT14,IGKV3-15,PMEL
1,1,LTB,ANXA1,NMB,LTB,CD79A,TCL1A,XCL1,C1QA,C1QB,...,LILRA4,FASLG,MS4A4A,DCN,COL1A2,TAGLN,RGS5,KRT17,IGHG1,MLANA
2,2,IL7R,CXCR4,NR3C1,IL32,CD83,MS4A1,XCL2,C1QC,C1QC,...,CD123,ID2,TNFSF8,PTGDS,COL3A1,MYL9,IGFBP7,LTF,IGKV1-5,MITF
3,3,SARAF,KLRB1,DUSP4,BATF,CD37,CD37,AREG,CCL3,CXCL9,...,TCF4,LAG3,CXCL10,CFD,LUM,TPM2,NDUFA4L2,KRT15,IGKV3-20,NaN
4,4,SELL,TNFAIP3,TNFRSF18,FOXP3,BANK1,CD74,KLRD1,CCL3L3,C1QA,...,IRF7,RGS1,EEF1B2,LUM,SFRP2,NDUFA4L2,CCL2,PTN,IGKV3-11,NaN


In [11]:
adata

AnnData object with n_obs × n_vars = 6492 × 17796
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [21]:
# Load expression counts and signature gene sets
adata, adata_normed = utils.load_adata(data_folder=data_path,
                                       sample_id=sample_id, # sample id
                                       n_genes=2000  # number of highly variable genes to keep
                                       )


[2024-03-24 04:05:59] Preprocessing1: delete the mt and rp
[2024-03-24 04:06:14] Preprocessing2: Normalize
[2024-03-24 04:06:14] Preprocessing3: Logarithm
[2024-03-24 04:06:14] Preprocessing4: Find the variable genes


In [22]:
# Load spatial information
img_metadata = utils.preprocess_img(data_path,
                                    sample_id,
                                    adata_index=adata.obs.index,
                                    hchannel=False
                                    )

FileNotFoundError: [Errno 2] No such file or directory: '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/alos/colon_work/adata/star_example/spatial/scalefactors_json.json'

Index(['AACAATCCGAGTGGAC-1', 'AACAATGTGCTCCGAG-1', 'AACACGACAACGGAGT-1',
       'AACACGACAATTGTTC-1', 'AACACGGAACGAGTTA-1', 'AACACGGCACCGATAG-1',
       'AACACGTTGATACCGC-1', 'AACACTCGTGAGCTTC-1', 'AACAGACCGCGATCTG-1',
       'AACAGCCTCCTGACTA-1',
       ...
       'TGTTGCCAGTCGCCTG-1', 'TGTTGCCGTTCGACCA-1', 'TGTTGCTATCGGTGGC-1',
       'TGTTGCTCGATGTCTG-1', 'TGTTGGAGCCGCGACG-1', 'TGTTGGATAGATCAGA-1',
       'TGTTGGCCGGATTGGT-1', 'TGTTGGTGATTAGGTA-1', 'TGTTGGTGCGGAATCA-1',
       'TGTTGGTGGACTCAGG-1'],
      dtype='object', length=6492)